In [2]:
import pandas as pd
import numpy as np
import os 
import re
pd.set_option('display.max_columns', None)

In [40]:
bets = pd.read_csv('../datasets/moneyline_w_outcomes.csv')
del bets['bet_won']
bets = bets.loc[bets.bet_type.isin(['Ordinary', 'Prop'])]
bets.head()

,bet_type,Bet,5Dimes,BetDSI,BookMaker,SportBet,Bet365,Bovada,Sportsbook,William_H,Pinnacle,SportsInt,BetOnline,Intertops,url,bet_both_sides_group,fighter1_clean,fighter2_clean,WeightClass,Outcome,Method,Round,Time,Notes,Card,Champion,Interim_Champion,wiki_url,Date,winner_clean,loser_clean
0,Ordinary,charles oliveira,2.170000,1.833333,2.140000,2.200000,2.250000,2.050000,2.150000,2.100000,2.290000,2.200000,2.100000,2.150000,https://www.bestfightodds.com/events/ufc-on-es...,0,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee
1,Ordinary,kevin lee,1.729927,1.833333,1.709220,1.746269,1.657895,1.740741,1.689655,1.729927,1.684932,1.645161,1.769231,1.689655,https://www.bestfightodds.com/events/ufc-on-es...,0,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee
2,Prop,Over 2½ rounds,2.200000,2.200000,2.310000,2.200000,NaN,NaN,NaN,2.200000,NaN,2.300000,2.250000,2.150000,https://www.bestfightodds.com/events/ufc-on-es...,1,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee
3,Prop,Under 2½ rounds,1.714286,1.709220,1.606061,1.714286,NaN,NaN,NaN,1.621118,NaN,1.588235,1.645161,1.689655,https://www.bestfightodds.com/events/ufc-on-es...,1,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee
4,Prop,Fight goes to decision,4.100000,NaN,NaN,4.100000,3.750000,NaN,NaN,4.000000,NaN,NaN,4.600000,NaN,https://www.bestfightodds.com/events/ufc-on-es...,2,charles oliveira,kevin lee,Catchweight (158.5 lbs),def.,Submission (guillotine choke),3.0,0:28,NaN,Main card (ESPN+/ESPN),NaN,NaN,https://en.wikipedia.org/wiki/UFC_Fight_Night:...,2020-03-14,charles oliveira,kevin lee


In [56]:
# get straight bets

finaldf_f1 = bets.loc[bets.fighter1_clean == bets.Bet, :].copy()
finaldf_f2 = bets.loc[bets.fighter2_clean == bets.Bet, :].copy()
betting_sites = ['5Dimes','BetDSI','BookMaker','SportBet','Bet365','Bovada', 
                'Sportsbook','William_H','Pinnacle','SportsInt','BetOnline','Intertops']

mergecol = list(bets.columns[~bets.columns.isin(betting_sites+['Bet'])])

straightbets = pd.merge(finaldf_f1, finaldf_f2, left_on = mergecol,
                   right_on = mergecol,
                   suffixes= ('_f1', '_f2'), validate = '1:1'
                  )
# propbets1 = bets.loc[bets.bet_type != 'Ordinary' , :].copy().groupby(['fighter1_clean', 'fighter2_clean', 'wiki_url', 'bet_both_sides_group']).nth(1).reset_index()
# propbets2 = bets.loc[bets.bet_type != 'Ordinary' , :].copy().groupby(['fighter1_clean', 'fighter2_clean', 'wiki_url', 'bet_both_sides_group']).nth(-1).reset_index()

# propbets = pd.merge(propbets1, propbets2, left_on = mergecol,
#                    right_on = mergecol,
#                    suffixes= ('_f1', '_f2'), validate = '1:1'
#                   )
# allbets = pd.concat([straightbets, propbets], sort = False)

In [57]:
for col in betting_sites:
    straightbets[col+"_vig"] = 1.0/straightbets[col+"_f1"] +  1.0/straightbets[col+"_f2"] -1.0
    if sum(straightbets[col+"_vig"] < 0):
        print(sum(straightbets[col+"_vig"] < 0) )

3
1
2


In [58]:
straightbets.to_csv(r"../datasets/straightbetwoutcomeslast7.csv", index = False)